In [ ]:
import numpy as np
from impedance import preprocessing
from impedance.models.circuits import CustomCircuit
import impedance.models.circuits.fitting
import numpy as np
from scipy.linalg import inv
from scipy.optimize import curve_fit, basinhopping

filename = "/Users/carypan/Documents/Laboratory/NiMH Project/Data/Powereign's Data/20210805_DCD35.csv"
data = np.genfromtxt(filename, delimiter = ',', dtype = None, skip_header = 0, encoding = 'UTF-8')
[Row,Column] = data.shape

fitdata = np.empty((Row, 16), dtype = object)
for i in range(6):
    data[0,i] = data[0,i].replace('"', '')
    fitdata[0,i] = data[0,i]

circuit = 'R1-p(R2,CPE1)-p(R3-Wo1,CPE2)'
initial_guess = [.01, .01, 100, .5, .01, .10**-3, .1, 100, .5]
circuit = CustomCircuit(circuit, initial_guess = initial_guess)
for i in range(len(circuit.get_param_names()[0])):
    fitdata[0,i+6] = circuit.get_param_names()[0][i] + ' / ' + circuit.get_param_names()[1][i]

for k in range(Row-1):
    frequencies = np.empty(int((Column-6)/3))
    Zreal = np.empty(int((Column-6)/3))
    Zimag = np.empty(int((Column-6)/3))
    for i in range(Column):
        data[k+1,i] = data[k+1,i].replace('"', '')
        if i < 6:
            fitdata[k+1,i] = data[k+1,i]
        else:
            if (i%3) == 0:
                frequencies[(int((i-6)/3))]=data[k+1,i]
            elif (i%3 == 1):
                Zreal[(int((i-6)/3))]=data[k+1,i]
            elif (i%3 == 2):
                Zimag[(int((i-6)/3))]=data[k+1,i]
    Z = Zreal + 1j*Zimag
    frequencies, Z = preprocessing.ignoreBelowX(frequencies, Z)
    circuit.fit(frequencies, Z)
    Z_fit = circuit.predict(frequencies)
    for i in range(len(circuit.get_param_names()[0])):
        fitdata[k+1,i+6] = circuit.parameters_[i]
    fitdata[k+1,15] = impedance.models.circuits.fitting.rmse(Z,Z_fit)